In [1]:
!pip install transformers
!pip install datasets

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/OpenSSL/crypto.py:14: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import utils, x509
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
     |████████████████████████████████| 389kB 8.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 28.9MB/s eta 0:00:01
     |████████████████████████████████| 389kB 27.5MB/s eta 0:00:01
     |████████████████████████████████| 512kB 28.8MB/s eta 0:00:01
     |████████████████████████████████| 890kB 23.7MB/s eta 0:00:01
     |████████████████████████████████| 81kB 16.6MB/s eta 0:00:01
     |████████████████████████████████| 7.7MB 17.4MB/s eta 0:00:01
  Us

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("esnli")

tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelWithLMHead.from_pretrained("t5-base")


Reusing dataset esnli (/Users/julianbruinsma/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc)
100%|██████████| 3/3 [00:00<00:00, 63.47it/s]
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:803: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Downloading: 100%|██████████| 850M/850M [00:21<00:00, 41.1MB/s] 


In [3]:
max_source_length = len(max(dataset['train']['premise'], key=len)) + len(max(dataset['train']['hypothesis'], key=len))
max_target_length = 512

In [4]:
train = dataset['train']
input_seqs = []
output_seqs = []
cnt = 0
for row in train:
    if cnt > 100000:
        break
    input_seqs.append(f"explain: premise: {row['premise']} hypothesis: {row['hypothesis']}</s>")
    output_seqs.append(row['explanation_1'])
    cnt += 1

In [11]:
encoding = tokenizer(input_seqs, padding='longest', max_length=max_source_length, truncation=True, return_tensors='pt')
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

target_encoding = tokenizer.encode(output_seqs, padding='longest', max_length=max_target_length, truncation=True, return_tensors='pt')
labels = target_encoding.input_ids
labels = torch.tensor(labels)
labels[labels == tokenizer.pad_token_id] = -100

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [10]:
test = dataset['test'][0]
input_seq = f"explain: premise: {test['premise']} hypothesis: {test['hypothesis']}</s>"
test_input = tokenizer.encode(input_seq, return_tensors='pt')
outputs = model.generate(test_input, num_beams=4, max_length=100, early_stopping=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Use model to predict a new string and print it 






neutral
